# Library

In [1]:
import os
from glob import glob
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import cv2
import matplotlib.pyplot as plt
from skimage import img_as_float
from torchvision import transforms
import os
from glob import glob
from sklearn.model_selection import train_test_split
import random
from collections import Counter, defaultdict

import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader

import os
from glob import glob
from sklearn.model_selection import train_test_split
import random
from collections import Counter, defaultdict
import cv2
import numpy as np
from torchvision import transforms
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
from torchvision import transforms

# --- Single Sequence Evaluation Setup ---

from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
from torchvision import transforms

# Load Dataset

In [2]:
def load_image_sequences(root_dir, stack_size=5):
    sequences = []
    for subject in os.listdir(root_dir):
        subject_path = os.path.join(root_dir, subject)
        if not os.path.isdir(subject_path):
            continue
        for label in os.listdir(subject_path):
            label_path = os.path.join(subject_path, label)
            if not os.path.isdir(label_path):
                continue
            for action in os.listdir(label_path):
                action_path = os.path.join(label_path, action)
                if not os.path.isdir(action_path):
                    continue
                image_paths = sorted(glob(os.path.join(action_path, '*.jpg')))
                # Pastikan cukup frame untuk stacking
                if len(image_paths) < stack_size:
                    continue
                # Ekstrak nama aksi tanpa angka depan jika ada
                action_name = "_".join(action.split('_')[1:]) if '_' in action else action
                sequences.append({
                    'subject': subject,
                    'label': label,
                    'action': action_name,
                    'image_paths': image_paths
                })
    return sequences

## Augmentation n Image Processing

In [3]:
augmentation = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
def apply_transformations(sequences, transform, stack_size=5):
    transformed_sequences = []
    for seq in sequences:
        transformed_images = []
        for img_path in seq['image_paths']:
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue  # Skip jika gambar tidak terbaca
            img = np.expand_dims(img, axis=2)  # Tambahkan dimensi channel
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)  # Ubah grayscale ke RGB
            img = np.array(img, dtype=np.uint8)
            augmented_img = transform(img)
            transformed_images.append(augmented_img)
        # Pastikan setelah transformasi, jumlah gambar masih >= stack_size
        if len(transformed_images) >= stack_size:
            transformed_sequences.append({
                'subject': seq['subject'],
                'label': seq['label'],
                'action': seq['action'],
                'image_tensors': transformed_images
            })
    return transformed_sequences

## Data Splitting

In [4]:
root_dir = 'train'  # Ganti dengan path sebenarnya
sequences = load_image_sequences(root_dir, stack_size=5)

# Memisahkan sequences berdasarkan label
fall_sequences = [s for s in sequences if s['label'] == 'fall']
non_fall_sequences = [s for s in sequences if s['label'] == 'non_fall']

# Membagi data masing-masing label menjadi train dan test dengan memastikan balance
train_fall, test_fall = train_test_split(fall_sequences, test_size=0.2, random_state=42)
train_non_fall, test_non_fall = train_test_split(non_fall_sequences, test_size=0.2, random_state=42)

# Menggabungkan train dan test set
train_sequences = train_fall + train_non_fall
test_sequences = test_fall + test_non_fall

random.seed(42)
random.shuffle(train_sequences)
random.shuffle(test_sequences)

# Mengacak urutan data
random.seed(42)
random.shuffle(train_sequences)
random.shuffle(test_sequences)

# Verifikasi distribusi label
train_labels = [s['label'] for s in train_sequences]
test_labels = [s['label'] for s in test_sequences]

print("Train label distribution:", Counter(train_labels))
print("Test label distribution:", Counter(test_labels))

In [6]:
def get_action_distribution(sequences):
    action_counts = defaultdict(int)
    action_subjects = defaultdict(set)
    for s in sequences:
        action = s['action']
        action_counts[action] += 1
        action_subjects[action].add(s['subject'])
    return action_counts, action_subjects

train_action_counts, train_action_subjects = get_action_distribution(train_sequences)
test_action_counts, test_action_subjects = get_action_distribution(test_sequences)

def print_action_distribution(action_counts, action_subjects, dataset_name):
    print(f"\n{dataset_name} Action Distribution:")
    print(f"{'Action':<30} {'Count':<10} {'Number of Subjects':<20}")
    print("-" * 60)
    for action, count in action_counts.items():
        num_subjects = len(action_subjects[action])
        print(f"{action:<30} {count:<10} {num_subjects:<20}")

print_action_distribution(train_action_counts, train_action_subjects, "Train")
print_action_distribution(test_action_counts, test_action_subjects, "Test")


Train Action Distribution:
Action                         Count      Number of Subjects  
------------------------------------------------------------
3_standing_falls               1          1                   
2_laying                       1          1                   
4_sitting_falls                1          1                   
4_walking                      1          1                   
2_stretching                   1          1                   
4_stretching                   1          1                   
4_left_falls                   1          1                   
1_left_falls                   1          1                   
3_jumping                      1          1                   
3_walking                      1          1                   
1_squat                        1          1                   
1_standing_falls               1          1                   
1_sitting_falls                1          1                   
2_squat                      

In [ ]:
# Terapkan augmentasi pada train_sequences
train_augmented = apply_transformations(train_sequences, augmentation, stack_size=5)

# Terapkan transformasi tanpa augmentasi pada test_sequences
test_transformed = apply_transformations(test_sequences, test_transform, stack_size=5)

# --- Stacking Consecutive Frames ---

def stack_consecutive_frames(sequences, stack_size=5):
    stacked_sequences = []
    for seq in sequences:
        images = seq['image_tensors']
        if len(images) < stack_size:
            continue
        for i in range(len(images) - stack_size + 1):
            stacked = torch.stack(images[i:i+stack_size])  # Shape: (stack_size, C, H, W)
            stacked_sequences.append({
                'subject': seq['subject'],
                'label': seq['label'],
                'action': seq['action'],
                'stacked_images': stacked
            })
    return stacked_sequences

# Terapkan stacking pada train_augmented
train_stacked = stack_consecutive_frames(train_augmented, stack_size=5)

# Terapkan stacking pada test_transformed
test_stacked = stack_consecutive_frames(test_transformed, stack_size=5)

# Model

In [9]:
class StackedActionDataset(Dataset):
    def __init__(self, stacked_sequences):
        self.stacked_sequences = stacked_sequences

    def __len__(self):
        return len(self.stacked_sequences)

    def __getitem__(self, idx):
        sequence = self.stacked_sequences[idx]
        images = sequence['stacked_images']  # Shape: [stack_size, C, H, W]
        label = 1 if sequence['label'] == 'fall' else 0
        # Rearrange to [C, stack_size, H, W] if model expects channels first
        images = images.permute(1, 0, 2, 3)  # [C, stack_size, H, W]
        return images, label

In [10]:
batch_size = 4  # Kurangi batch size untuk menghindari penggunaan memori berlebih

train_dataset = StackedActionDataset(train_stacked)
test_dataset = StackedActionDataset(test_stacked)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [11]:
class Conv3DModel(nn.Module):
    def __init__(self, pretrained=True):
        super(Conv3DModel, self).__init__()
        # Menggunakan model 3D ResNet18 sebagai backbone
        self.backbone = models.video.r3d_18(pretrained=pretrained)
        num_ftrs = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()  # Menghapus layer akhir

        # Menambahkan layer klasifikasi
        self.classifier = nn.Sequential(
            nn.Linear(num_ftrs, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 2)  # Kelas: fall dan non_fall
        )

    def forward(self, x):
        # x shape: [batch_size, C, stack_size, H, W]
        x = self.backbone(x)  # Output shape: [batch_size, num_ftrs]
        x = self.classifier(x)  # Output shape: [batch_size, 2]
        return x

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Conv3DModel(pretrained=True).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

c:\Users\ASUS\anaconda3\envs\venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ASUS\anaconda3\envs\venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/r3d_18-b3b3357e.pth" to C:\Users\ASUS/.cache\torch\hub\checkpoints\r3d_18-b3b3357e.pth
100%|██████████| 127M/127M [00:37<00:00, 3.57MB/s] 


# Training

In [13]:
num_epochs = 10
    
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images = images.to(device)  # Shape: [batch_size, C, stack_size, H, W]
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct_train / total_train
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

# --- Evaluation ---
    
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
print(f'Accuracy of the model on test sequences: {100 * correct / total:.2f}%')

RuntimeError: Given groups=1, weight of size [64, 3, 3, 7, 7], expected input[8, 5, 3, 224, 224] to have 3 channels, but got 5 channels instead

## Evaluation

# Testing

In [ ]:
# --- Modified Evaluation Function to Stack the Same Frame ---
    
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
from torchvision import transforms
import torch
from collections import deque

class SingleFrameDataset(Dataset):
    def __init__(self, sequences, transform=None):
        """
        Dataset for single frame evaluation.

        Args:
            sequences (list): List of sequences from the test set.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.frame_paths = []
        self.labels = []
        for seq in sequences:
            for img_path in seq['image_paths']:
                self.frame_paths.append(img_path)
                self.labels.append(1 if seq['label'] == 'fall' else 0)
        self.transform = transform

    def __len__(self):
        return len(self.frame_paths)

    def __getitem__(self, idx):
        img_path = self.frame_paths[idx]
        label = self.labels[idx]

        # Load image in grayscale
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            raise ValueError(f"Image not found or unable to read: {img_path}")

        # Convert grayscale to RGB
        img = np.expand_dims(img, axis=2)  # [H, W, 1]
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)  # [H, W, 3]
        img = np.array(img, dtype=np.uint8)

        if self.transform:
            img = self.transform(img)  # [C, H, W]

        return img, label

# Define transformations similar to training
single_frame_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Sesuaikan untuk RGB
                         std=[0.229, 0.224, 0.225])
])

# Create SingleFrameDataset
single_frame_test_dataset = SingleFrameDataset(test_sequences, transform=single_frame_transform)

# Create DataLoader with batch_size=1 dan tanpa shuffle
single_frame_test_loader = DataLoader(single_frame_test_dataset, batch_size=1, shuffle=False)

# --- Modified Evaluation Function ---

def evaluate_single_frame(model, single_frame_loader, device, stack_size=5):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(single_frame_loader):
            # Debugging: Cetak informasi batch
            print(f'Processing batch {batch_idx + 1}')

            # images: [batch_size=1, C, H, W]
            images = images.to(device)
            labels = labels.to(device)

            frame = images[0]  # [C, H, W]

            # Membuat stack dengan mengulangi frame yang sama
            stacked_frames = frame.unsqueeze(1).repeat(1, stack_size, 1, 1)  # [C, stack_size, H, W]

            # Menambahkan dimensi batch: [1, C, stack_size, H, W]
            stacked_frames = stacked_frames.unsqueeze(0)

            # Debugging: Cetak bentuk tensor
            print(f'Stacked Frames Shape: {stacked_frames.shape}')  # Harusnya [1, C, stack_size, H, W]

            # Pengecekan dimensi
            assert stacked_frames.dim() == 5, f"Expected 5D tensor, got {stacked_frames.dim()}D tensor"

            # Forward pass
            outputs = model(stacked_frames)
            _, predicted = torch.max(outputs, 1)

            # Debugging: Tampilkan prediksi dan label
            print(f'Predicted: {predicted.item()}, Actual: {labels.item()}')

            total += 1
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total if total > 0 else 0
    print(f'Accuracy of the model on single frame test sequences: {accuracy:.2f}%')

# --- Modified Evaluation Call ---

evaluate_single_frame(model, single_frame_test_loader, device, stack_size=5)